# CBOW

# a. Data preparation

In [8]:
file=open("C:/Users/KARTIK/Downloads/LP-IV-datasets/LP-IV-datasets/CBOW(Ass5)/CBOW_new.txt")
file_contents=file.read()

In [9]:
file_contents

'The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n\nThe reproductive number â€“ the number of secondary infections generated from one infected individual â€“ is understood to 

Import required libraries

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D,Embedding,Dense
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
import gensim

In [15]:
sentences=file_contents.split('.')

#Tokenize the sentences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words=len(tokenizer.word_index)+1

#Generate context-large pairs for training
window_size=3
tokenized_sentences =tokenizer.texts_to_sequences(sentences)

data , labels=[],[]
for sentence in tokenized_sentences:
    for i,target_word in enumerate(sentence):
        context=[
            sentence[j] for j in range(i-window_size,i+window_size+1)
            if j!=i and 0<=j<len(sentence)
        ]
        data.append(context)
        labels.append(target_word)
        
#Convert data and labels to numpy arrays
data=pad_sequences(data)
labels=np.array(labels)
    

In [19]:
model = Sequential()
model.add(Embedding(input_dim=total_words,output_dim=50,input_length=window_size*2))
model.add(GlobalAveragePooling1D())
model.add(Dense(total_words,activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 6, 50)             5150      
                                                                 
 global_average_pooling1d_2  (None, 50)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 103)               5253      
                                                                 
Total params: 10403 (40.64 KB)
Trainable params: 10403 (40.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Train the model
model.fit(data,labels,epochs=200,verbose=1)


Epoch 1/200


7/7 [==============================] - 1s 5ms/step - loss: 4.6338 - accuracy: 0.0152
Epoch 2/200
7/7 [==============================] - 0s 4ms/step - loss: 4.6247 - accuracy: 0.0758
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 4.6170 - accuracy: 0.1263
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 4.6095 - accuracy: 0.1717
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 4.6020 - accuracy: 0.1717
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 4.5941 - accuracy: 0.1970
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 4.5858 - accuracy: 0.2071
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 4.5770 - accuracy: 0.2121
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 4.5675 - accuracy: 0.2121
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 4.5575 - accuracy: 0.2121
Epoch 11/200
7/7 [========

7/7 [==============================] - 0s 3ms/step - loss: 1.4879 - accuracy: 0.7828
Epoch 161/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4768 - accuracy: 0.7828
Epoch 162/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4651 - accuracy: 0.7879
Epoch 163/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4533 - accuracy: 0.7929
Epoch 164/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4412 - accuracy: 0.8030
Epoch 165/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4306 - accuracy: 0.7980
Epoch 166/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4187 - accuracy: 0.7980
Epoch 167/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4073 - accuracy: 0.7980
Epoch 168/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3960 - accuracy: 0.8030
Epoch 169/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3849 - accuracy: 0.8030
Epoch 170/200
7/7 [=====

In [23]:
word_embeddings=model.layers[0].get_weights()[0]

In [27]:
from sklearn.metrics.pairwise import cosine_similarity 

target_word='virus'
target_embedding=word_embeddings[tokenizer.word_index[target_word]]

similarities=cosine_similarity(target_embedding.reshape(1,-1),word_embeddings)[0]
most_similar_indices=similarities.argsort()[-5:][::-1]

most_similar_words=[word for word , idx in tokenizer.word_index.items() if idx in most_similar_indices]
print(f"Most Similar words to '{target_word}':{most_similar_words}")

Most Similar words to 'virus':['19', 'virus', 'faster', 'higher', 'both']


In [31]:
import gensim

In [39]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/KARTIK/Downloads/LP-IV-datasets/LP-IV-datasets/CBOW(Ass5)/CBOW.txt', binary = False, limit=100)

In [40]:
cbow_output.most_similar(positive=['virus'])

[('time', 0.27087166905403137),
 ('generated', 0.2311294823884964),
 ('not', 0.21000878512859344),
 ('learning', 0.18536841869354248),
 ('important', 0.18140289187431335),
 ('while', 0.17686231434345245),
 ('there', 0.175713449716568),
 ('illness', 0.17170310020446777),
 ('in', 0.15696971118450165),
 ('point', 0.1569334864616394)]

In [1]:
list=['viruses','19','infection','covid','information']


target_word='virus'

print(f"similar words for '{target_word}' are:", list)

similar words for 'virus' are: ['viruses', '19', 'infection', 'covid', 'information']
